In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"
os.environ["NCCL_P2P_DISABLE"] = "1"

In [2]:
import torch
from datasets import load_dataset, load_metric,concatenate_datasets
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments,AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.utils.data import DataLoader
from tqdm import tqdm
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from datasets import Dataset, DatasetDict, load_dataset, load_metric
import torch
import random

In [3]:
def check_requires_grad(model):
    print("Checking which high-level components require gradients:\n")

    # Checking Embeddings
    print("Embeddings:")
    for name, param in model.wrapped_model.bert.embeddings.named_parameters():
        grad_status = "requires gradient" if param.requires_grad else "does NOT require gradient"
        print(f"  {name} - {grad_status}")

    # Checking Encoder
    print("\nEncoder:")
    for i, layer in enumerate(model.wrapped_model.bert.encoder.layer):
        print(f"  Layer {i}:")
        for name, param in layer.named_parameters():
            grad_status = "requires gradient" if param.requires_grad else "does NOT require gradient"
            print(f"    {name} - {grad_status}")

    # Checking Attention
    print("\nAttention:")
    for i, layer in enumerate(model.wrapped_model.bert.encoder.layer):
        print(f"  Layer {i} Attention:")
        for name, param in layer.attention.named_parameters():
            grad_status = "requires gradient" if param.requires_grad else "does NOT require gradient"
            print(f"    {name} - {grad_status}")

    # Checking MLM Head
    print("\nMasked LM Head:")
    for name, param in model.wrapped_model.cls.named_parameters():
        grad_status = "requires gradient" if param.requires_grad else "does NOT require gradient"
        print(f"  {name} - {grad_status}")

    print("\nCheck complete.")


test the relora experiments---------
test trainable scaling



test bigger bert
maybe there is a mistake in the training loop, fix it ----------
try warmup or bigger model
try using not mlm but normal causal language modeling (change the model initialization + collator etc)
the form_pretrian methosd may be wrong but i don't use it rn.
check the dataloader
check in the original reelora file if lora_a is 0--

problems:
less time for training
higher perplexity

check hyperparameters
compare with lora training loop

In [4]:
#chocolate-galaxy-701
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path bert_c4 \
    --dataset_path /home/delyanh/Projects/ReVeRA/preprocessed_cola_dataset_BERT_MEDIUM \
    --batch_size 64\
    --lr 1e-3\
    --max_length 512 \
    --weight_decay 0.02\
    --num_training_steps 62500 \
    --save_every 600000\
    --eval_every 3500 \
    --warmup_steps 9000 \
    --seed 42\
    --tags relora_bert_medium \
    --workers 4 \
    --use_double_quant False \
    --use_peft True \
    --cycle_length 15625 \
    --restart_warmup_steps 500 \
    --scheduler cosine_restarts \
    --reset_optimizer_on_relora False \
    --relora 15625\
    --lora_r 64\
    --optimizer_magnitude_pruning 0.9\
    --lora_alpha 64\
    


2024-09-21 22:49:33.271 | INFO     | __main__:main:369 - Global rank 0, local rank 0, device: 0
2024-09-21 22:49:33.272 | INFO     | __main__:main:373 - Process group initialized
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /home/delyanh/Projects/ReVeRA/ReLoRa/wandb/run-20240921_224934-9c92utk5
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run graceful-snowball-734
wandb: ⭐️ View project at https://wandb.ai/delyanhristov06-/peft_pretraining
wandb: 🚀 View run at https://wandb.ai/delyanhristov06-/peft_pretraining/runs/9c92utk5
2024-09-21 22:49:35.690 | INFO     | __main__:main:443 - Using dist with rank 0 (only rank 0 will log)
2024-09-21 22:49:35.690 | INFO     | __main__:main:444 - **************************************

# TEST

In [ ]:
-min_lr_ratio 0.000000000000000000000000000000000000000000001\

In [5]:
#chocolate-galaxy-701
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path bert_c4 \
    --dataset_path /home/delyanh/Projects/ReVeRA/preprocessed_cola_dataset_BERT_MEDIUM \
    --batch_size 64\
    --lr 5e-4 \
    --max_length 512 \
    --weight_decay 0.03 \
    --num_training_steps 62500 \
    --save_every 600000\
    --eval_every 3000 \
    --warmup_steps 9000 \
    --seed 42 \
    --tags relora_bert_medium \
    --workers 4 \
    --use_double_quant False \
    --use_peft False \
    --cycle_length 15625 \
    --restart_warmup_steps 500 \
    --scheduler linear \
    --reset_optimizer_on_relora False \
    --relora 15625\
    --lora_r 64\
    --optimizer_magnitude_pruning 0.9\
    --lora_alpha 64 \
    --min_lr_ratio 0.000000000000000000000000000000000000000000001 

2024-09-22 00:49:29.312 | INFO     | __main__:main:369 - Global rank 0, local rank 0, device: 0
2024-09-22 00:49:29.314 | INFO     | __main__:main:373 - Process group initialized
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /home/delyanh/Projects/ReVeRA/ReLoRa/wandb/run-20240922_004930-ptz0xf8p
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run royal-snow-735
wandb: ⭐️ View project at https://wandb.ai/delyanhristov06-/peft_pretraining
wandb: 🚀 View run at https://wandb.ai/delyanhristov06-/peft_pretraining/runs/ptz0xf8p
2024-09-22 00:49:31.268 | INFO     | __main__:main:443 - Using dist with rank 0 (only rank 0 will log)
2024-09-22 00:49:31.268 | INFO     | __main__:main:444 - ****************************************
2024

In [6]:
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path bert_c4 \
    --dataset_path /home/delyanh/Projects/ReVeRA/preprocessed_cola_dataset_BERT_MEDIUM \
    --batch_size 64\
    --lr 5e-4 \
    --max_length 512 \
    --weight_decay 0.02 \
    --num_training_steps 62500 \
    --save_every 600000\
    --eval_every 3000 \
    --warmup_steps 9000 \
    --seed 42 \
    --tags relora_bert_medium \
    --workers 4 \
    --use_double_quant False \
    --use_peft True \
    --cycle_length 62500 \
    --restart_warmup_steps 500 \
    --scheduler linear \
    --reset_optimizer_on_relora False \
    --relora 62500\
    --lora_r 64\
    --optimizer_magnitude_pruning 0.9\
    --lora_alpha 128 \
    --min_lr_ratio 0.000000000000000000000000000000000000000000001 

2024-09-22 02:38:40.220 | INFO     | __main__:main:369 - Global rank 0, local rank 0, device: 0
2024-09-22 02:38:40.221 | INFO     | __main__:main:373 - Process group initialized
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /home/delyanh/Projects/ReVeRA/ReLoRa/wandb/run-20240922_023841-yzbe6as6
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run lunar-terrain-736
wandb: ⭐️ View project at https://wandb.ai/delyanhristov06-/peft_pretraining
wandb: 🚀 View run at https://wandb.ai/delyanhristov06-/peft_pretraining/runs/yzbe6as6
2024-09-22 02:38:42.190 | INFO     | __main__:main:443 - Using dist with rank 0 (only rank 0 will log)
2024-09-22 02:38:42.190 | INFO     | __main__:main:444 - ****************************************
2

In [7]:
#chocolate-galaxy-701
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path bert_c4 \
    --dataset_path /home/delyanh/Projects/ReVeRA/preprocessed_cola_dataset_BERT_MEDIUM \
    --batch_size 64\
    --lr 1e-3\
    --max_length 512 \
    --weight_decay 0.02\
    --num_training_steps 62500 \
    --save_every 600000\
    --eval_every 3500 \
    --warmup_steps 9000 \
    --seed 42\
    --tags relora_bert_medium \
    --workers 4 \
    --use_double_quant False \
    --use_peft True \
    --cycle_length 15625 \
    --restart_warmup_steps 500 \
    --scheduler cosine_restarts \
    --reset_optimizer_on_relora False \
    --relora 15625\
    --lora_r 64\
    --optimizer_magnitude_pruning 0.9\
    --lora_alpha 64\
    


2024-09-22 04:38:21.152 | INFO     | __main__:main:369 - Global rank 0, local rank 0, device: 0
2024-09-22 04:38:21.154 | INFO     | __main__:main:373 - Process group initialized
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /home/delyanh/Projects/ReVeRA/ReLoRa/wandb/run-20240922_043822-tnd7kgy2
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run desert-durian-737
wandb: ⭐️ View project at https://wandb.ai/delyanhristov06-/peft_pretraining
wandb: 🚀 View run at https://wandb.ai/delyanhristov06-/peft_pretraining/runs/tnd7kgy2
2024-09-22 04:38:23.127 | INFO     | __main__:main:443 - Using dist with rank 0 (only rank 0 will log)
2024-09-22 04:38:23.127 | INFO     | __main__:main:444 - ****************************************
2

In [8]:
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path bert_c4 \
    --dataset_path /home/delyanh/Projects/ReVeRA/preprocessed_cola_dataset_BERT_MEDIUM \
    --batch_size 64\
    --lr 6e-4 \
    --max_length 512 \
    --weight_decay 0.02 \
    --num_training_steps 62500 \
    --save_every 600000\
    --eval_every 3000 \
    --warmup_steps 9000 \
    --seed 42 \
    --tags relora_bert_medium \
    --workers 4 \
    --use_double_quant False \
    --use_peft True \
    --cycle_length 62500 \
    --restart_warmup_steps 500 \
    --scheduler linear \
    --reset_optimizer_on_relora False \
    --relora 62500\
    --lora_r 64\
    --optimizer_magnitude_pruning 0.9\
    --lora_alpha 128 \
    --min_lr_ratio 0.000000000000000000000000000000000000000000001 

2024-09-22 06:37:17.143 | INFO     | __main__:main:369 - Global rank 0, local rank 0, device: 0
2024-09-22 06:37:17.145 | INFO     | __main__:main:373 - Process group initialized
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /home/delyanh/Projects/ReVeRA/ReLoRa/wandb/run-20240922_063718-mibktbuq
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run smart-surf-738
wandb: ⭐️ View project at https://wandb.ai/delyanhristov06-/peft_pretraining
wandb: 🚀 View run at https://wandb.ai/delyanhristov06-/peft_pretraining/runs/mibktbuq
2024-09-22 06:37:19.049 | INFO     | __main__:main:443 - Using dist with rank 0 (only rank 0 will log)
2024-09-22 06:37:19.049 | INFO     | __main__:main:444 - ****************************************
2024

In [9]:
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path bert_c4 \
    --dataset_path /home/delyanh/Projects/ReVeRA/preprocessed_cola_dataset_BERT_MEDIUM \
    --batch_size 64\
    --lr 6e-4 \
    --max_length 512 \
    --weight_decay 0.02 \
    --num_training_steps 62500 \
    --save_every 600000\
    --eval_every 3000 \
    --warmup_steps 9000 \
    --seed 42 \
    --tags relora_bert_medium \
    --workers 4 \
    --use_double_quant False \
    --use_peft True \
    --cycle_length 62500 \
    --restart_warmup_steps 500 \
    --scheduler linear \
    --reset_optimizer_on_relora False \
    --relora 62500\
    --lora_r 64\
    --optimizer_magnitude_pruning 0.9\
    --lora_alpha 64 \
    --min_lr_ratio 0.000000000000000000000000000000000000000000001 

2024-09-22 08:36:58.119 | INFO     | __main__:main:369 - Global rank 0, local rank 0, device: 0
2024-09-22 08:36:58.120 | INFO     | __main__:main:373 - Process group initialized
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /home/delyanh/Projects/ReVeRA/ReLoRa/wandb/run-20240922_083659-mn2c40ra
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run classic-fog-739
wandb: ⭐️ View project at https://wandb.ai/delyanhristov06-/peft_pretraining
wandb: 🚀 View run at https://wandb.ai/delyanhristov06-/peft_pretraining/runs/mn2c40ra
2024-09-22 08:37:00.037 | INFO     | __main__:main:443 - Using dist with rank 0 (only rank 0 will log)
2024-09-22 08:37:00.037 | INFO     | __main__:main:444 - ****************************************
202

In [10]:
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path bert_c4 \
    --dataset_path /home/delyanh/Projects/ReVeRA/preprocessed_cola_dataset_BERT_MEDIUM \
    --batch_size 64\
    --lr 6e-4 \
    --max_length 512 \
    --weight_decay 0.02 \
    --num_training_steps 62500 \
    --save_every 600000\
    --eval_every 3000 \
    --warmup_steps 9000 \
    --seed 42 \
    --tags relora_bert_medium \
    --workers 4 \
    --use_double_quant False \
    --use_peft True \
    --cycle_length 62500 \
    --restart_warmup_steps 500 \
    --scheduler linear \
    --reset_optimizer_on_relora False \
    --relora 62500\
    --lora_r 64\
    --optimizer_magnitude_pruning 0.9\
    --lora_alpha 64 \
    --min_lr_ratio 0.000000000000000000000000000000000000000000001 

2024-09-22 10:36:44.051 | INFO     | __main__:main:369 - Global rank 0, local rank 0, device: 0
2024-09-22 10:36:44.053 | INFO     | __main__:main:373 - Process group initialized
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /home/delyanh/Projects/ReVeRA/ReLoRa/wandb/run-20240922_103644-0yhz6j2s
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run expert-feather-740
wandb: ⭐️ View project at https://wandb.ai/delyanhristov06-/peft_pretraining
wandb: 🚀 View run at https://wandb.ai/delyanhristov06-/peft_pretraining/runs/0yhz6j2s
2024-09-22 10:36:45.969 | INFO     | __main__:main:443 - Using dist with rank 0 (only rank 0 will log)
2024-09-22 10:36:45.969 | INFO     | __main__:main:444 - ****************************************


In [11]:
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path bert_c4 \
    --dataset_path /home/delyanh/Projects/ReVeRA/preprocessed_cola_dataset_BERT_MEDIUM \
    --batch_size 64\
    --lr 4e-4 \
    --max_length 512 \
    --weight_decay 0.02 \
    --num_training_steps 62500 \
    --save_every 600000\
    --eval_every 3000 \
    --warmup_steps 9000 \
    --seed 42 \
    --tags relora_bert_medium \
    --workers 4 \
    --use_double_quant False \
    --use_peft True \
    --cycle_length 62500 \
    --restart_warmup_steps 500 \
    --scheduler linear \
    --reset_optimizer_on_relora False \
    --relora 62500\
    --lora_r 64\
    --optimizer_magnitude_pruning 0.9\
    --lora_alpha 128 \
    --min_lr_ratio 0.000000000000000000000000000000000000000000001 

2024-09-22 12:36:30.023 | INFO     | __main__:main:369 - Global rank 0, local rank 0, device: 0
2024-09-22 12:36:30.025 | INFO     | __main__:main:373 - Process group initialized
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /home/delyanh/Projects/ReVeRA/ReLoRa/wandb/run-20240922_123630-tw48i0jz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run deft-sponge-741
wandb: ⭐️ View project at https://wandb.ai/delyanhristov06-/peft_pretraining
wandb: 🚀 View run at https://wandb.ai/delyanhristov06-/peft_pretraining/runs/tw48i0jz
2024-09-22 12:36:32.055 | INFO     | __main__:main:443 - Using dist with rank 0 (only rank 0 will log)
2024-09-22 12:36:32.055 | INFO     | __main__:main:444 - ****************************************
202

compare with standart lora to see if everything is working

after that test fewer warm_up steps
different magnitude pruning